In [17]:
import torch
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer, pipeline
from datasets import load_dataset
import evaluate
from tqdm import tqdm

In [3]:
data_files = {"train": "../data/Dataset1/train_sentiment.csv", "validation": "../data/Dataset1/val_sentiment.csv", "test": "../data/Dataset1/test_sentiment.csv"}
ds = load_dataset("csv", data_files=data_files, sep="\t")
ds

DatasetDict({
    train: Dataset({
        features: ['review/score', 'review/text'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['review/score', 'review/text'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['review/score', 'review/text'],
        num_rows: 1000
    })
})

In [4]:
# rename the score column to labels to align with hf dataset format
ds = ds.rename_column("review/score", "labels")
ds

DatasetDict({
    train: Dataset({
        features: ['labels', 'review/text'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['labels', 'review/text'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['labels', 'review/text'],
        num_rows: 1000
    })
})

In [4]:
# load in the tokenizer to preprocess the review text field
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# preprocessing function to tokenize text and truncate sentence to not more than BERT maximum input length
def preprocess_function(examples):
    return tokenizer(examples["review/text"], truncation=True)

# apply the preprocessing function, batch for faster mapping
tokenized_ds = ds.map(preprocess_function, batched=True)

# pad the text in each review to the longest length in a batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 1000/1000 [00:01<00:00, 565.08 examples/s]


In [14]:
# define evaluation function and metrics during training
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    return accuracy.compute(predictions=predictions, references=labels)

In [6]:
# before training, map the expected ids to the labels (1.0 review to negative, 5.0 review to positive)
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [13]:
# Define BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2, id2label=id2label, label2id=label2id)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Freeze all layers except the final classification layer
for param in model.bert.parameters():
    param.requires_grad = False

training_args = TrainingArguments(
    output_dir="BERT_sentiment",
    optim="adamw_torch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=15,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.684833,0.581000
2,No log,0.678984,0.620000
3,No log,0.673925,0.628000
4,0.683100,0.669881,0.620000
5,0.683100,0.666433,0.634000
6,0.683100,0.663132,0.644000
7,0.683100,0.660349,0.649000
8,0.665700,0.657855,0.655000
9,0.665700,0.655758,0.671000
10,0.665700,0.654347,0.665000


TrainOutput(global_step=1875, training_loss=0.66638291015625, metrics={'train_runtime': 1584.1846, 'train_samples_per_second': 37.874, 'train_steps_per_second': 1.184, 'total_flos': 1.547777094260736e+16, 'train_loss': 0.66638291015625, 'epoch': 15.0})

In [ ]:
# conduct inference on test set

model_path = r"\Users\sorui\Desktop\NLPExplorationProject\Application-1\BERT_sentiment\checkpoint-1875" # change to your model path
model = BertForSequenceClassification.from_pretrained(model_path) 
tokenizer = BertTokenizer.from_pretrained(model_path) 

predictions = []
references = []

for i in tqdm(range(len(ds["test"]["labels"])), desc = "Generating predictions"):
    references.append(ds["test"]["labels"][i])
    inputs = tokenizer(ds["test"]["review/text"][i], return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    predictions.append(predicted_class_id)
    
recall = evaluate.load('recall')
precision = evaluate.load("precision")
f1 = evaluate.load("f1")

print(accuracy.compute(predictions=predictions, references=references), precision.compute(predictions=predictions, references=references),
      recall.compute(predictions=predictions, references=references), f1.compute(predictions=predictions, references=references))

Generating predictions: 100%|██████████| 1000/1000 [01:27<00:00, 11.44it/s]

{'accuracy': 0.671} {'precision': 0.6451612903225806} {'recall': 0.76} {'f1': 0.6978879706152433}
